<a href="https://colab.research.google.com/github/envirodatascience/final-project-utility-solar-on-home-prices/blob/main/Utility_Solar_and_Home_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

As the United States moves towards decarbonizing the energy system, utility scale solar will become more widespread due to its affordability and production of minimal-to-zero emissions. The Solar Energy Industries Association finds that there are more than 37,000 megawatts of utility-scale solar projects in operation, with another 112,000 megawatts being developed.

Given the growing prominence of utility scale solar, our group wanted to better understand the geographic context of the communities that they are sited near and explore utility scale solar's impacts on the housing market. We seek to answer the following question:

***Does the existence of nearby (1 mile) utility scale solar impact property values at the neighborhood level?***

To answer this question, we will use the following data:


* *United States Large-Scale Solar Photovoltaic Database (USPVDB)*

  This database contains information on the locations and boundaries of U.S. PV facilities with a capacity of 1 megawatt or greater.
```
  https://eerscmap.usgs.gov/uspvdb/assets/data/uspvdbSHP.zip
```

* *Zillow Home Value Index (ZHVI)*

  This data presents the typical values for homes within the 65th to 95th percentile range for a given region. The data is smoothed and seasonally adjusted, and does not account for inflation.
```
 https://files.zillowstatic.com/research/public_csvs/zhvi/Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1712011928
```

* *Neighborhood Boundaries*

 This data contains boundaries for 17,000 Zillow neighborhoods in the United States.
```
 https://edg.epa.gov/data/PUBLIC/OEI/ZILLOW_NEIGHBORHOODS/Zillow_Neighborhoods.zip
```

* *United States Boundaries*

  This data contains boundaries for all states in the United States.

  ```
  https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip
  ```




# Part 1: Importing Packages

In [ ]:
# basics
import pandas as pd
import numpy as np

# geo
import geopandas as gpd
from shapely.geometry import Polygon

# plotting
from plotnine import *
import plotnine

# misc
from IPython.display import display
pd.options.display.max_columns = None
import statsmodels.api as sm
import scipy.stats as stats

# Part 2: Download and Unzip Data

## Part 2.1: United States Large-Scale Solar Photovoltaic Database (USPVDB)

In [ ]:
! wget https://eerscmap.usgs.gov/uspvdb/assets/data/uspvdbSHP.zip

In [ ]:
! ls

In [ ]:
! unzip uspvdbSHP.zip

Archive:  uspvdbSHP.zip
replace CHANGELOG.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! ls

In [ ]:
solar = 'uspvdb_v1_0_20231108.shp'
df_solar = gpd.read_file(solar)

In [ ]:
# Check to see if data was downloaded
df_solar.head()

## Part 2.2: Zillow Home Value Index (ZHVI)

In [ ]:
url = 'https://files.zillowstatic.com/research/public_csvs/zhvi/Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1712011928'
df_homes = pd.read_csv(url)

In [ ]:
# Check to see if data was downloaded
df_homes.head()

## Part 2.3: Neighborhood Boundaries

In [ ]:
! wget https://edg.epa.gov/data/PUBLIC/OEI/ZILLOW_NEIGHBORHOODS/Zillow_Neighborhoods.zip

In [ ]:
! ls

In [ ]:
! unzip Zillow_Neighborhoods.zip

In [ ]:
! ls

In [ ]:
neighborhoods = 'ZillowNeighborhoods.gdb'
df_neighborhoods = gpd.read_file(neighborhoods)

In [ ]:
# Check if data downloaded
df_neighborhoods.head()

## Part 2.4: United States Boundaries

In [ ]:
! wget https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip

In [ ]:
! ls

In [ ]:
! unzip cb_2018_us_state_20m.zip

In [ ]:
! ls

In [ ]:
states = 'cb_2018_us_state_20m.shp'
df_states = gpd.read_file(states)

In [ ]:
# Check if data downloaded
df_states.head(5)

# Part 3: Read and Analyze Data

## Part 3.1: Analyzing Solar PV Data

Overview of the Section:
*   How many counties are represented?
*   How many states are represented
* How big are the largest PVs, and where are they located?
* Which states have the largest area of PVs?
*  Are there any duplicates?
* Preliminary plot



In [ ]:
df_solar.info()

In [ ]:
df_solar.describe()

# The largest solar PV area represented by the data is 1.52 x 10^7 square meters, and the
# smallest solar PV area represented is 1.56 x 10^3 square meters.

In [ ]:
df_solar.head()

**How many counties are represented?**

In [ ]:
df_solar.p_county.nunique()

In [ ]:
df_solar.p_county.unique()

In [ ]:
df_solar.groupby('p_county').agg(n_obs = ('p_county', 'count')).sort_values(['n_obs'], ascending = False)

# Worcester, Kern, and Los Angeles counties have the most recorded solar PVs

**How many states are represented?**

In [ ]:
df_solar.p_state.nunique()

In [ ]:
df_solar.p_state.unique()

In [ ]:
df_solar.groupby('p_state').agg(n_obs = ('p_state', 'count')).sort_values(['n_obs'], ascending = False)

# North Carolina, California, and Minnesota are the top three states
# with the most solar PVs.

**How big are the largest PVs, and where are they located?**

In [ ]:
df_solar.sort_values('p_area', ascending = False).head(5)

# The top five largest PVs represented by the data are in California and Texas.

**Which states have the largest area of PVs?**

In [ ]:
df_solar.groupby('p_state').agg(total_area = ('p_area', 'sum')).sort_values('total_area', ascending = False).reset_index()

# California, Texas, and North Carolina have the largest area of PVs.

**Are there any duplicates in the data?**

In [ ]:
df_solar[df_solar.duplicated()]

Preliminary Plot

In [ ]:
df_solar.plot()

## Part 3.2: Analyzing Home Value Data

Overview of the Section:
* What years does this data cover?
*   How many counties are represented?
* Where are the most expensive homes?
*   How many states are represented
*  Are there any duplicates?

In [ ]:
df_homes.info()

In [ ]:
df_homes.head()

In [ ]:
df_homes.describe()

**How many counties are represented?**

In [ ]:
df_homes.CountyName.nunique()

In [ ]:
df_homes.CountyName.unique()

In [ ]:
df_homes.groupby('CountyName').agg(n_obs = ('CountyName', 'count')).sort_values(['n_obs'], ascending = False)

# Maricopa, Tarrant, and Harris counties are the most represented by the data.

**Where are the most expensive homes?**

In [ ]:
df_homes.groupby('CountyName').agg(mean_price = ('2024-01-31', 'mean')).reset_index().sort_values('mean_price', ascending = False)

# Walton County, Santa Clara County, and Collier County have the most expensive home prices

**How many states are represented?**

In [ ]:
df_homes.State.nunique()

In [ ]:
df_homes.groupby('State').agg(n_obs = ('State', 'count')).sort_values(['n_obs'], ascending = False)

# Texas, California, and Florida are the top three states represented by the data.

**Are there any duplicates in the data?**

In [ ]:
df_homes[df_homes.duplicated()]

## Part 3.3: Checking Neighborhood Data

Overview of the Section:
*   How many counties are represented?
*   How many states are represented
* Which states have the most surface area represented?
*  Are there any duplicates?
* Preliminary plot


In [ ]:
df_neighborhoods.info()

In [ ]:
df_neighborhoods.describe()

In [ ]:
df_neighborhoods.head()

**How many counties are represented?**

In [ ]:
df_neighborhoods.County.nunique()

In [ ]:
df_neighborhoods.County.unique()

In [ ]:
df_neighborhoods.groupby('County').agg(n_obs = ('County', 'count')).sort_values(['n_obs'], ascending = False)

# Davidson, Maricopa, and Bexar counties are the most represented in the data.

**How many states are represented?**

In [ ]:
df_neighborhoods.State.nunique()

In [ ]:
df_neighborhoods.State.unique()

In [ ]:
df_neighborhoods.groupby('State').agg(n_obs = ('State', 'count')).sort_values(['n_obs'], ascending = False)

# California, Texas, and Florida are the top three states most represented by the data.

**Which states have the most surface area represented?**

In [ ]:
df_neighborhoods.groupby('State').agg(total_area = ('Shape_Area', 'sum')).sort_values('total_area', ascending = False).reset_index()

# California, Arkansas, and Texas have the largest neighborhood areas in the data

**Are there any duplicates in the data?**

In [ ]:
df_neighborhoods[df_neighborhoods.duplicated()]

Preliminary Plot

In [ ]:
df_neighborhoods.plot()

# Part 4: Selecting Neighborhoods and Solar PV that Intersect

## Part 4.1: Checking Coordinate Reference Systems

In [ ]:
df_neighborhoods.crs

In [ ]:
df_solar.crs

In [ ]:
df_states.crs

## Part 4.2: Converting CRS

In [ ]:
df_neighborhoods = df_neighborhoods.to_crs("ESRI:102003")
df_neighborhoods.crs

In [ ]:
df_states = df_states.to_crs("ESRI:102003")
df_states.crs

In [ ]:
plotnine.options.figure_size = (15, 6)


(ggplot()
 + geom_map (df_states, fill="none", color = "black")
 + geom_map(df_solar, color='red')
 + geom_map(df_neighborhoods, color = 'blue')
 + theme_classic()
 + theme(axis_line=element_line(color="white"),
          axis_ticks=element_line(color = "white"),
          axis_text=element_line(color='white'),
          text=element_text(size = 12))
)

## Part 4.3: Buffer Solar PVs

In this section we will create a 1 mile buffer around the solar PVs. Any panel buffer and neighborhood that intersect will be utilized in our analysis, all others will be dropped out of the analysis.

Since our coordinate system is displayed in meters, 1 mile is equal to 1609.34 meters. We will use this to specify the distance of our buffer.

In [ ]:
df_solar_2 = df_solar

In [ ]:
df_solar_2['buffer'] = df_solar_2.buffer(1609.34)

In [ ]:
df_solar_2.head()

In [ ]:
df_buffer = df_solar_2[['buffer', 'case_id']].copy()
df_buffer['geometry'] = df_buffer['buffer']
df_buffer.head()

In [ ]:
plotnine.options.figure_size = (15, 6)


(ggplot()
 + geom_map (df_states, fill="none", color = "black")
 + geom_map(df_buffer, color='#112c5e')
 + geom_map(df_neighborhoods, color = 'red')
 + theme_classic()
 + theme(axis_line=element_line(color="white"),
          axis_ticks=element_line(color = "white"),
          axis_text=element_line(color='white'),
          text=element_text(size = 12))
)

## Part 4.4: Selecting Solar PVs and Neighborhoods that Intersect

In [ ]:
df_join = gpd.sjoin(df_buffer, df_neighborhoods, how='inner', predicate='intersects')
df_join.head()

In [ ]:
df_join.info()

**Selecting Solar PVs that Intersect**

In [ ]:
df_solar_select = df_solar[df_solar['case_id'].isin((df_join['case_id']))]
df_solar_select.head(5)

In [ ]:
df_solar_select.info()

In [ ]:
df_join['case_id'].nunique()

# In the join table there were only 252 unique case_ids. This is why the resulting solar_select only has 252 rows.
# This means that there are some solar PVs that overlap with multiple neighborhoods.

**Selecting Neighborhoods that Intersect**

In [ ]:
df_neigh_PV = df_neighborhoods[df_neighborhoods['RegionID'].isin((df_join['RegionID']))]
df_neigh_PV.head(5)

In [ ]:
df_neigh_PV.info()

In [ ]:
df_join.RegionID.nunique()

In [ ]:
df_neigh_PV.RegionID.nunique()

## Part 4.5: Selecting Neighborhoods without PV


In [ ]:
df_neigh_no_PV = df_neighborhoods[~df_neighborhoods['RegionID'].isin((df_join['RegionID']))]
df_neigh_no_PV.head(5)

## Part 4.6: Preliminary Plots

In [ ]:
# Figuring out which states have neighborhoods within 1 mile of solar PVs
df_neigh_PV.State.nunique()

In [ ]:
df_neigh_PV.State.unique()

In [ ]:
# Removing Alaska, Hawaii, and Puerto Rico (no neighborhoods in proximity) to improve size of map

df_contiguous = df_states[~((df_states.NAME == 'Hawaii') | (df_states.NAME == 'Puerto Rico') | (df_states.NAME == 'Alaska'))]

In [ ]:
plotnine.options.figure_size = (18, 6)

(ggplot()
 + geom_map (df_contiguous, fill="none", color = "black")
 + geom_map(df_neigh_PV, color = 'orange')
 + geom_map(df_solar_select, color='#112c5e')
 + theme_classic()
 + theme(axis_line=element_line(color="white"),
          axis_ticks=element_line(color = "white"),
          axis_text=element_line(color='white'),
          text=element_text(size = 12))
 + ggtitle ("Neighborhoods in Within 1 Mile Proximity to Solar Panels")
)

In [ ]:
plotnine.options.figure_size = (15, 6)

(ggplot()
 + geom_map (df_states, fill="none", color = "black")
 + geom_map(df_neigh_no_PV, color = 'red')
 + geom_map(df_solar_select, color='#112c5e')
 + theme_classic()
 + theme(axis_line=element_line(color="white"),
          axis_ticks=element_line(color = "white"),
          axis_text=element_line(color='white'),
          text=element_text(size = 12))
 + ggtitle ("Neighborhoods NOT Within 1 Mile Proximity to Solar Panels")
)

In [ ]:
df_homes.head(5)

# Part 5: Analyzing Changes in Home Prices

Check and confirm the DataFrame structure and date columns. Let's make sure we use a method to safely identify and work with the date columns:

In [ ]:
date_columns = [col for col in df_homes.columns if pd.to_datetime(col, errors='coerce') is not pd.NaT]

## 5.1: Three-year home value CAGR calculations

We are using a 3-year compound annual growth rate (CAGR) as a metric to compare before and after growth rates for a given year. A 3-year period was chosen in order to capture delayed effects that might not be noticeable in just 1 or 2 years. Longer time period would both limit the availability of data points (since we are constrained by just having ~20 years of data) and could end up capturing other long term dynamic of the housing market. CAGR was chosen for ease of understanding the results (annualized rates are more common than 3-year % changes)


Calculate CAGR for every 3-year period where applicable:

In [ ]:
for year in range(2001, 2022):
    start_col = f"{year}-12-31"
    end_col = f"{year+3}-12-31"

    # Ensure columns are in the DataFrame before attempting to access them
    if start_col in date_columns and end_col in date_columns:
        # Calculate CAGR for the 3-year period:
        # Make sure both columns contain numerical data by confirming their presence in date_columns
        start_values = df_homes[start_col]
        end_values = df_homes[end_col]

        # Compute the CAGR:
        df_homes[f'{year}-{year+3} CAGR'] = (end_values / start_values) ** (1/3) - 1

Display the DataFrame to check new columns

In [ ]:
df_homes.head()

Manual calculation for check

In [ ]:
df_homes['2001-2004 CAGR v2'] = ( df_homes['2004-12-31'] / df_homes['2001-12-31'] )**(1/3) -1

Check that manual calculation and for loop match. False seems to come from rows with NaN values

In [ ]:
check = df_homes['2001-2004 CAGR v2'] == df_homes['2001-2004 CAGR']
check.head(40)

In [ ]:
df_homes.head(29)

Here we have a check to see if the NaN were coming from an error in code or underlying data (seeing it it shows up in both the for look version and the manual calculation) Print the value using label index:

In [ ]:
value = df_homes.loc[28]['2001-2004 CAGR']
print("Value in row 28 and column '2001-2004 CAGR':", value)

In [ ]:
value = df_homes.loc[28]['2001-2004 CAGR v2']
print("Value in row 28 and column '2001-2004 CAGR v2':", value)

## 5.2: Year-on-year home value growth rate calculations

Here we calculate year-on-year growth rates for the difference-in-difference analysis. It makes more sense to look at annual changes one at a time (vs. changes in 3-year changes)  

In [ ]:
# Year-on-year growth rate for diff in diff calculaion
# Calculate year-on-year for each year:
for year in range(2001, 2022):
    start_col = f"{year}-12-31"
    end_col = f"{year+1}-12-31"

    # Ensure columns are in the DataFrame before attempting to access them
    if start_col in date_columns and end_col in date_columns:
        # Calculate Y-o-Y growth rate for each each:
        # Make sure both columns contain numerical data by confirming their presence in date_columns
        start_values = df_homes[start_col]
        end_values = df_homes[end_col]

        # Compute the annual change as a %:
        df_homes[f'{year}_YoY_change'] = (end_values / start_values) - 1

# Display the DataFrame to check new columns
df_homes.head()

# Part 6: Evaluating the Impacts of Solar Development on Home Values over Time

Here, we seek to evaluate how the 3-year period growth rates of home values differed between solar and non-solar neighborhoods over time.

## Part 6.1: Determine Pre- and Post- CAGR For Solar Neighborhoods

In [ ]:
# convert nominal neighborhood Region ID to string
df_homes['RegionID'] = df_homes['RegionID'].astype('str')

In [ ]:
# merge neighborhood CAGR and solar PV selection dataframes - this pairs each solar project with its intersecting neighborhood's yearly CAGR data
df_join2 = df_join.merge(df_homes, on = 'RegionID', how = 'inner')
df_solar_homes = df_join2.merge(df_solar_select, on = 'case_id', how = 'inner')

In [ ]:
# initialize empty dataframe
df_solar_homes_cagr = pd.DataFrame()

# for each year in the dataset, create a temporary dataframe containing only panels installed that year with its corresponding difference
# in pre- and post- CAGR values
for year in df_solar_homes['p_year'].unique():
  try:
    df_temp = df_solar_homes[df_solar_homes['p_year'] == year].copy()
    df_temp['pre_CAGR'] = df_temp[f'{year-3}-{year} CAGR']
    df_temp['post_CAGR'] = df_temp[f'{year}-{year+3} CAGR']
    df_temp['diff_CAGR'] = df_temp['post_CAGR'] - df_temp['pre_CAGR']
    df_temp['type'] = 'solar'
    df_solar_homes_cagr = pd.concat([df_solar_homes_cagr, df_temp]) # concatenate yearly CAGR difference data to global dataframe
  except:
    pass

df_solar_homes_cagr = df_solar_homes_cagr[['case_id', 'p_year', 'diff_CAGR', 'type']]

## Part 6.2: Determine Pre- and Post- CAGR For Non-Solar Neighborhoods

In [ ]:
# merge neighborhood CAGR and no-solar PV selection dataframes - this narrows down the neighborhood CAGR to only those without PV projects nearby
df_no_solar_homes = df_neigh_no_PV.merge(df_homes, on = 'RegionID', how = 'inner')
df_no_solar_homes.head(5)

In [ ]:
# initialize empty dataframe
df_no_solar_homes_cagr = pd.DataFrame()

# for each year, create a temporary dataframe containing difference in pre- and post- CAGR values for every region ID, for every year
for year in range(2001, 2022):
  try:
    df_temp = df_no_solar_homes
    df_temp['p_year'] = year
    df_temp['pre_CAGR'] = df_temp[f'{year-3}-{year} CAGR']
    df_temp['post_CAGR'] = df_temp[f'{year}-{year+3} CAGR']
    df_temp['diff_CAGR'] = df_temp['post_CAGR'] - df_temp['pre_CAGR']
    df_temp['type'] = 'no solar'
    df_no_solar_homes_cagr = pd.concat([df_no_solar_homes_cagr, df_temp])
  except:
    pass

df_no_solar_homes_cagr = df_no_solar_homes_cagr[['RegionID', 'p_year', 'diff_CAGR', 'type']]

## Part 6.3: Statistical Analysis

This section seeks to evaluate the statistical distributions and values of the solar and non-solar CAGR distributions.

In [ ]:
# combine the solar and non-solar CAGR data
df_homes_cagr = pd.concat([df_solar_homes_cagr, df_no_solar_homes_cagr])

In [ ]:
# aggreate data to show average difference in CAGR per year, per solar presence; also show count of neighborhoods included in calculation
df_homes_cagr.groupby(['p_year', 'type']).agg(avg_cagr = ('diff_CAGR', 'mean'),
                                              n = ('p_year', 'count')).reset_index()

In [ ]:
#calcuate the t-statistic for each year's solar and non-solar CAGR difference values
for year in df_homes_cagr['p_year'].unique():
  solar = df_homes_cagr[(df_homes_cagr['p_year'] == year) & (df_homes_cagr['type'] == 'solar')]['diff_CAGR'].dropna()
  non_solar = df_homes_cagr[(df_homes_cagr['p_year'] == year) & (df_homes_cagr['type'] == 'no solar')]['diff_CAGR'].dropna()
  print(f'{year} statistics: {stats.ttest_ind(solar, non_solar)}')

For the years 2007, 2012, 2016, 2017, and 2020 (5/14 years) there was a statistically significant (P < 0.05) difference in the three-year CAGR for solar versus non-solar neighborhoods. This means in these years, home values in neighborhoods with nearby solar developments increased OR decreased at significantly different rates than those in non-solar adjacent neighborhoods.

## Part 6.4: Box-and-Whisker Plot

In [ ]:
# convert p_year to categorical for plotting purposes
df_homes_cagr['p_year'] = pd.Categorical(df_homes_cagr['p_year'])

# plot CAGR distribution boxplots, separated by year and solar presence
(
  ggplot(df_homes_cagr, aes(x="p_year", y="diff_CAGR", fill = 'type'))
  + geom_boxplot()
  + xlab("Year")
  + ylab("Difference in CAGR")
  + labs(fill = 'Neighborhood')
  + ggtitle("Difference in Neighborhood Home Price CAGR - Aggregated by Presence of Nearby Solar")
    + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),
    )
    + scale_fill_manual(values=("red", "orange"))
)

This time series box plot graph shows how the CAGR distribution for solar and non-solar neighborhoods over time. Early in the series (2004-2009), the difference in CAGR were generally negative, meaning home prices were falling. This makes sense particularly for 2006, which takes into account the financial crisis of 2008-2009. In these years, the post-2006 three-year period (which incldues 2009) has a very negative CAGR; meanwhile the pre-2006 three-year period may have a normal/neutral CAGR value. As a result, the difference in CAGR for 2006 would be negative.

Over the years, the database has an increasing number of solar developments, which is reflected in the appearance of solar-adjacent neighborhoods in the graph. Overall, there is an increasing trend in CAGR difference following the financial crisis.

Averages of solar neighborhoods generally tended to be higher than the non-solar neighborhoods, but not by too much. Between neighborhood types, averages are very similar, with averages falling within the 25-75th percentiles of each other. While the t-tests showed statistically significant differences in a few of these years, visually the difference is not large relative to the entire plot.


# Part 7: Graphing the Impact of Solar Development on Home Values over Time: Difference-in-Difference



**Overview of the Section:**

Find average AGR for all neighborhoods within 1 mile of a solar development project for the 3 years preceeding and post treatment (Solar Development Installation), for the 3 years of highest development (2014, 2017, and 2018)

Find average AGR for all neighborhoods ***not*** within 1 mile of a solar development project for the 3 years preceeding and post the 3 years of highest development (2014, 2017, and 2018). This will be our non-treatment group.

Plot these data points, and analyze any difference-in-differences between the two data points (Solar Developed and Solar not-Developed) pre- and post- installation for between 2010-2018.

## Part 7.1 Isolating solar development projects by year

In [ ]:
# df_solar_select (All the Solar Development Projects Built within 1 mile of a Neighborhood on Zillow)
  # df_solar_select_2014 (All the Solar Development Projects Built within 1 mile of a Neighborhood on Zillow in 2014)
  # df_solar_select_2017 (All the Solar Development Projects Built within 1 mile of a Neighborhood on Zillow in 2017)
  # df_solar_select_2018 (All the Solar Development Projects Built within 1 mile of a Neighborhood on Zillow in 2018)

# df_no_solar_homes (All Neighborhoods not within 1 mile of a Solar Development Project)


In [ ]:
df_solar_homes["p_year"].value_counts()
#.value_counts is used here to see what the expected number of values and which years will have a statistically robust enough sample.
#from this filtering we can see that prior to 2010, there were very few eligible neighborhoods, so (2009, 2008, and 2007 will be removed from the sample, 2019, 2020, and 2021 will also
#be removed as we only have YoY data that is complete through 2021, and thus would have incomplete post-installation data for these three years)

In [ ]:
# create individual call codes filtering for each of the years for use later in the process
df_solar_select_2010 = df_solar_homes[df_solar_homes['p_year'] == 2010]
df_solar_select_2011 = df_solar_homes[df_solar_homes['p_year'] == 2011]
df_solar_select_2012 = df_solar_homes[df_solar_homes['p_year'] == 2012]
df_solar_select_2013 = df_solar_homes[df_solar_homes['p_year'] == 2013]
df_solar_select_2014 = df_solar_homes[df_solar_homes['p_year'] == 2014]
df_solar_select_2015 = df_solar_homes[df_solar_homes['p_year'] == 2015]
df_solar_select_2016 = df_solar_homes[df_solar_homes['p_year'] == 2016]
df_solar_select_2017 = df_solar_homes[df_solar_homes['p_year'] == 2017]
df_solar_select_2018 = df_solar_homes[df_solar_homes['p_year'] == 2018]

## Part 7.2 Create new data frame with the mean Year over Year growth rate for the three years pre and post solar project development.      

In [ ]:
# Use .mean to find the mean of each of the YoY Growth Rates
# Create new Data Frame that has these stats for each of the 6 years (3 prior, 3 post)


In [ ]:
#Solar Homes: YoY

#2010
dd_agr_2k10 = ["2007_YoY_change","2008_YoY_change","2009_YoY_change","2010_YoY_change","2011_YoY_change","2012_YoY_change","2013_YoY_change"]
df_dd_agr_2k10 = df_solar_select_2010[dd_agr_2k10].mean()

#2011
dd_agr_2k11 = ["2008_YoY_change","2009_YoY_change","2010_YoY_change","2011_YoY_change","2012_YoY_change","2013_YoY_change","2014_YoY_change"]
df_dd_agr_2k11 = df_solar_select_2011[dd_agr_2k11].mean()

#2012
dd_agr_2k12 = ["2009_YoY_change","2010_YoY_change","2011_YoY_change","2012_YoY_change","2013_YoY_change","2014_YoY_change","2015_YoY_change"]
df_dd_agr_2k12 = df_solar_select_2012[dd_agr_2k12].mean()

#2013
dd_agr_2k13 = ["2010_YoY_change","2011_YoY_change","2012_YoY_change","2013_YoY_change","2014_YoY_change","2015_YoY_change","2016_YoY_change"]
df_dd_agr_2k13 = df_solar_select_2013[dd_agr_2k13].mean()

#2014
dd_agr_2k14 = ["2011_YoY_change","2012_YoY_change","2013_YoY_change","2014_YoY_change","2015_YoY_change","2016_YoY_change","2017_YoY_change"]
df_dd_agr_2k14 = df_solar_select_2014[dd_agr_2k14].mean()

#2015
dd_agr_2k15 = ["2012_YoY_change","2013_YoY_change","2014_YoY_change","2015_YoY_change","2016_YoY_change","2017_YoY_change","2018_YoY_change"]
df_dd_agr_2k15 = df_solar_select_2015[dd_agr_2k15].mean()

#2016
dd_agr_2k16 = ["2013_YoY_change","2014_YoY_change","2015_YoY_change","2016_YoY_change","2017_YoY_change","2018_YoY_change","2019_YoY_change"]
df_dd_agr_2k16 = df_solar_select_2016[dd_agr_2k16].mean()

#2017
dd_agr_2k17 = ["2014_YoY_change","2015_YoY_change","2016_YoY_change","2017_YoY_change","2018_YoY_change","2019_YoY_change","2020_YoY_change"]
df_dd_agr_2k17= df_solar_select_2017[dd_agr_2k17].mean()

#2018
dd_agr_2k18 = ["2015_YoY_change","2016_YoY_change","2017_YoY_change","2018_YoY_change","2019_YoY_change","2020_YoY_change","2021_YoY_change"]
df_dd_agr_2k18 = df_solar_select_2018[dd_agr_2k18].mean()


In [ ]:
# No Solar Homes

#2010
dd_nosolar2k10 = ["2007_YoY_change","2008_YoY_change","2009_YoY_change","2010_YoY_change","2011_YoY_change","2012_YoY_change","2013_YoY_change"]
df_dd_nosolar2k10 =  df_no_solar_homes[dd_nosolar2k10].mean()

#2011
dd_nosolar2k11 = ["2008_YoY_change","2009_YoY_change","2010_YoY_change","2011_YoY_change","2012_YoY_change","2013_YoY_change","2014_YoY_change"]
df_dd_nosolar2k11 = df_no_solar_homes[dd_nosolar2k11].mean()

#2012
dd_nosolar2k12 = ["2009_YoY_change","2010_YoY_change","2011_YoY_change","2012_YoY_change","2013_YoY_change","2014_YoY_change","2015_YoY_change"]
df_dd_nosolar2k12 = df_no_solar_homes[dd_nosolar2k12].mean()

#2013
dd_nosolar2k13 = ["2010_YoY_change","2011_YoY_change","2012_YoY_change","2013_YoY_change","2014_YoY_change","2015_YoY_change","2016_YoY_change"]
df_dd_nosolar2k13 = df_no_solar_homes[dd_nosolar2k13].mean()

#2014
dd_nosolar2k14 = ["2011_YoY_change","2012_YoY_change","2013_YoY_change","2014_YoY_change","2015_YoY_change","2016_YoY_change","2017_YoY_change"]
df_dd_nosolar2k14 = df_no_solar_homes[dd_nosolar2k14].mean()

#2015
dd_nosolar2k15 = ["2012_YoY_change","2013_YoY_change","2014_YoY_change","2015_YoY_change","2016_YoY_change","2017_YoY_change","2018_YoY_change"]
df_dd_nosolar2k15 = df_no_solar_homes[dd_nosolar2k15].mean()

#2016
dd_nosolar2k16 = ["2013_YoY_change","2014_YoY_change","2015_YoY_change","2016_YoY_change","2017_YoY_change","2018_YoY_change","2019_YoY_change"]
df_dd_nosolar2k16 = df_no_solar_homes[dd_nosolar2k16].mean()

#2017
dd_nosolar2k17 = ["2014_YoY_change","2015_YoY_change","2016_YoY_change","2017_YoY_change","2018_YoY_change","2019_YoY_change","2020_YoY_change"]
df_dd_nosolar2k17= df_no_solar_homes[dd_nosolar2k17].mean()

#2018
dd_nosolar2k18 = ["2015_YoY_change","2016_YoY_change","2017_YoY_change","2018_YoY_change","2019_YoY_change","2020_YoY_change","2021_YoY_change"]
df_dd_nosolar2k18 = df_no_solar_homes[dd_nosolar2k18].mean()


In [ ]:
# Now for each of the parsed out data frames for the different years,
#we will add a column that has the average for each of
#the three years preceeding and post target date of the "treatment" (p_year), and make this into a new merged and melted data frame ready for graphing

In [ ]:
#combine these data points into a single dataframe, along with the year for easiest graphing, data tidiness, and ensuring the legend shows up properly.
#2010
NoSolar2k10 = pd.DataFrame(list(df_dd_nosolar2k10.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k10 = pd.DataFrame(list(df_dd_agr_2k10.items()), columns = ['Year', "Solar_AGR"])
df_2k10 = pd.merge(NoSolar2k10, Solar2k10, on= "Year" , how= "outer")
df_2k10["Year"] = df_2k10["Year"].str.replace("_YoY_change", "")
df_2k10['Year'] = pd.to_numeric(df_2k10["Year"])

#2011
NoSolar2k11 = pd.DataFrame(list(df_dd_nosolar2k11.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k11 = pd.DataFrame(list(df_dd_agr_2k11.items()), columns = ['Year', "Solar_AGR"])
df_2k11 = pd.merge(NoSolar2k11, Solar2k11, on= "Year" , how= "outer")
df_2k11["Year"] = df_2k11["Year"].str.replace("_YoY_change", "")
df_2k11['Year'] = pd.to_numeric(df_2k11["Year"])

#2012
NoSolar2k12 = pd.DataFrame(list(df_dd_nosolar2k12.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k12 = pd.DataFrame(list(df_dd_agr_2k12.items()), columns = ['Year', "Solar_AGR"])
df_2k12 = pd.merge(NoSolar2k12, Solar2k12, on= "Year" , how= "outer")
df_2k12["Year"] = df_2k12["Year"].str.replace("_YoY_change", "")
df_2k12['Year'] = pd.to_numeric(df_2k12["Year"])

#2013
NoSolar2k13 = pd.DataFrame(list(df_dd_nosolar2k13.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k13 = pd.DataFrame(list(df_dd_agr_2k13.items()), columns = ['Year', "Solar_AGR"])
df_2k13 = pd.merge(NoSolar2k13, Solar2k13, on= "Year" , how= "outer")
df_2k13["Year"] = df_2k13["Year"].str.replace("_YoY_change", "")
df_2k13['Year'] = pd.to_numeric(df_2k13["Year"])

#2014
NoSolar2k14 = pd.DataFrame(list(df_dd_nosolar2k14.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k14 = pd.DataFrame(list(df_dd_agr_2k14.items()), columns = ['Year', "Solar_AGR"])
df_2k14 = pd.merge(NoSolar2k14, Solar2k14, on= "Year" , how= "outer")
df_2k14["Year"] = df_2k14["Year"].str.replace("_YoY_change", "")
df_2k14['Year'] = pd.to_numeric(df_2k14["Year"])

#2015
NoSolar2k15 = pd.DataFrame(list(df_dd_nosolar2k15.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k15 = pd.DataFrame(list(df_dd_agr_2k15.items()), columns = ['Year', "Solar_AGR"])
df_2k15 = pd.merge(NoSolar2k15, Solar2k15, on= "Year" , how= "outer")
df_2k15["Year"] = df_2k15["Year"].str.replace("_YoY_change", "")
df_2k15['Year'] = pd.to_numeric(df_2k15["Year"])

#2016
NoSolar2k16 = pd.DataFrame(list(df_dd_nosolar2k16.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k16 = pd.DataFrame(list(df_dd_agr_2k16.items()), columns = ['Year', "Solar_AGR"])
df_2k16 = pd.merge(NoSolar2k16, Solar2k16, on= "Year" , how= "outer")
df_2k16["Year"] = df_2k16["Year"].str.replace("_YoY_change", "")
df_2k16['Year'] = pd.to_numeric(df_2k16["Year"])

#2017
NoSolar2k17 = pd.DataFrame(list(df_dd_nosolar2k17.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k17 = pd.DataFrame(list(df_dd_agr_2k17.items()), columns = ['Year', "Solar_AGR"])
df_2k17 = pd.merge(NoSolar2k18, Solar2k18, on= "Year" , how= "outer")
df_2k17["Year"] = df_2k17["Year"].str.replace("_YoY_change", "")
df_2k17['Year'] = pd.to_numeric(df_2k17["Year"])

#2018

NoSolar2k18 = pd.DataFrame(list(df_dd_nosolar2k18.items()), columns = ['Year', "No_Solar_AGR"])
Solar2k18 = pd.DataFrame(list(df_dd_agr_2k18.items()), columns = ['Year', "Solar_AGR"])
df_2k18 = pd.merge(NoSolar2k18, Solar2k18, on= "Year" , how= "outer")
df_2k18["Year"] = df_2k18["Year"].str.replace("_YoY_change", "")
df_2k18['Year'] = pd.to_numeric(df_2k18["Year"])


In [ ]:
#melt the data frames such that they are able to be graphed utilizing the aes feature

df_2k10melt = df_2k10.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')
df_2k11melt = df_2k11.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')
df_2k12melt = df_2k12.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')
df_2k13melt = df_2k13.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')
df_2k14melt = df_2k14.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')
df_2k15melt = df_2k15.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')
df_2k16melt = df_2k16.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')
df_2k17melt = df_2k17.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')
df_2k18melt = df_2k18.melt(id_vars=['Year'], value_vars=['Solar_AGR','No_Solar_AGR'], var_name='aes', value_name='AGR')

## 7.3 Plotting Results (Difference-in-Difference Graphs)

In [ ]:
# Use plotnine graphing package (ggplot) to plot a multiple line graph that plots the data from the dataframes created in subsections 7.2
# add a dashed vertical line at the "treatement date" (Date of Project Implementation)

In [ ]:
#2010
(
ggplot () +
  geom_line(df_2k10melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2010, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2010, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

In [ ]:
#2011
(
ggplot () +
  geom_line(df_2k11melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2011, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2011, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

In [ ]:
#2012
(
ggplot () +
  geom_line(df_2k12melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2012, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2012, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

In [ ]:
#2013
(
ggplot () +
  geom_line(df_2k13melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2013, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2013, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

In [ ]:
#2014
(
ggplot () +
  geom_line(df_2k14melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2014, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2014, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

In [ ]:
#2015
(
ggplot () +
  geom_line(df_2k15melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2015, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2015, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

In [ ]:
#2016
(
ggplot () +
  geom_line(df_2k16melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2016, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2016, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

In [ ]:
#2017
(
ggplot () +
  geom_line(df_2k17melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2017, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2017, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

In [ ]:
#2018
(
ggplot () +
   geom_line(df_2k18melt, aes(x="Year", y="AGR", color="aes")) # AES variable set to the merged category column (Solar/Non-Solar)

 + geom_vline (xintercept=2018, linetype="dashed") # Add Project Date Line

    +scale_color_manual(values = {"No_Solar_AGR" : "red", "Solar_AGR" : "orange"})  # Define colors

+ xlab("Year")
  + ylab("Annual Growth Rate (Year over Year %)")
    + labs(color = 'Solar Construction')

    + ggtitle("Difference in Difference - Average AGR for Solar Projects Constructed in 2018, Compared to Non-Solar Proximate Home AGR's")
   + theme(
        axis_line=element_line(size=1, colour="black"),
        panel_grid_major=element_line(colour="#d3d3d3"),
        panel_grid_minor=element_blank(),
        panel_border=element_blank(),
        panel_background=element_blank(),)
    )

# Part 8: Conclusion

### Is There An Impact On Property Values?
In the analysis conducted above, we explored whether the existence of nearby (within 1 mile) utility scale solar impacted property values at the neighborhood level. Overall, we found that there is a neutral to slightly positive impact on the growth of home prices, though the evidence of change is underwhelming. Across 14 years of data in the Box-and-Whisker Plot, there are five years that are statisticaly significant in the change in property values across solar and non-solar adjacent neighborhoods. Moreover, general trends in the Difference-in-Difference plots demonstrated that  property value growth rate changes were not drastically different between solar and non-solar adjacent neighborhoods.

### Follow-Up Questions
After completing the analysis, we are left with the following questions:
*   Does the area/capacity of the utility scale solar impact property values at the neighborhood scale?
*   Is there a more pronounced effect in conservative versus progressive leaning areas? Is one political affilitation more critical about utility scale solar than the other?
* Would neighborhoods containing larger proportion of each solar developments' viewshed exhibit different CAGR patterns? Currently, we are using Euclidean buffer distances.
* What would be the effect of utility scale solar in less urbanized areas?

### Furthering the Analysis
While our analysis indicates a potential neutral to positive impact, the extent of our analysis is restricted by data assumptions and limitations. One of these is assumptions is that we chose an arbitrary 1 mile buffer around utility scale solar to define neighborhood that are "close" versus not close to solar. Moreover, we rely on the neighborhoods found in the Zillow neighborhood data - these tend to be found in metro areas.

With these assumptions and limitations in mind, and time-permitting, we would further the analysis by creating multple buffer distances from the utility scale solar (i.e. 1 mile, 5 miles, 10 miles, 15 miles) and determine whether the size of the buffer had any impacts on growth rates. Additionally, we would hope to incorporate a viewshed analysis to determine whether the utility scale solar is visible to the neighborhoods that we are analyzing. Finally, because the Zillow data focuses more on metro-area neighborhoods, we could incorporate neighborhoods located in rural areas into our analysis.